In [ ]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from matplotlib.projections.geo import GeoAxes
import astropy.units as u

# Healpy osa
nside = 64

gal_l = wake_coords.galactic.l.wrap_at(180 * u.deg)
gal_b = wake_coords.galactic.b

lon = gal_l.radian
lat = gal_b.radian

theta = 0.5*np.pi - lat                # latitude -> colatitude
phi   = lon % (2*np.pi)                # wrap to [0, 2π)

pix = hp.ang2pix(nside, theta, phi)
m = np.bincount(pix, minlength=hp.nside2npix(nside)).astype(float)

# Overdensity osa
pop = m > 0
mu  = m[pop].mean() if np.any(pop) else 1.0

delta = np.full_like(m, hp.UNSEEN, dtype=float)
delta[pop] = (m[pop] / mu) - 1.0

# Smoothing funktsioon, proovi ilma selleta! See 5 on siin suvaline, seda peab vaatama veel. 
fwhm = np.radians(5)
delta_sm = hp.smoothing(delta, fwhm=fwhm, verbose=False)

# Healpy -> Matplib toimub siin
def project_healpy_to_plt(nside, xsize=2000):
    """Return (grid_pix, xsize, ysize) for Matplotlib Mollweide plotting."""
    ysize = xsize // 2                   # Mollweide aspect 2:1
    theta_grid = np.linspace(np.pi, 0, ysize)
    phi_grid   = np.linspace(-np.pi, np.pi, xsize)
    PHI, THETA = np.meshgrid(phi_grid, theta_grid)
    grid_pix   = hp.ang2pix(nside, THETA, PHI)
    return grid_pix, xsize, ysize

# Pmst. milline healpy pixel iga matplotlibi pikslile vastab
grid_pix, xsize, ysize = project_healpy_to_plt(nside)
grid_map = delta_sm[grid_pix]           # fancy indexing -> (ysize, xsize)

longitude = np.radians(np.linspace(-180, 180, xsize))
latitude  = np.radians(np.linspace(-90,   90, ysize))

# Korrastab ticke
class ThetaFormatterShiftPi(GeoAxes.ThetaFormatter):
    """Shift labels from (-180,180) → (0–360) and flip sign for astro convention."""
    def __call__(self, x, pos=None):
        if x != 0:
            x *= -1
        if x < 0:
            x += 2*np.pi
        return GeoAxes.ThetaFormatter.__call__(self, x, pos)

#
# Siin on alles plottimine
#

fig = plt.figure(figsize=(10, 5), facecolor='white')
ax = fig.add_subplot(111, projection="mollweide")

# Neid vmin/vmax väärtusi võib vajaduse muuta, Foote-is on -0.4 kuni 1.0
valid = np.isfinite(grid_map)
vmax = np.nanpercentile(grid_map[valid], 99)
vmax = float(max(vmax, 0.1))            # avoid zero; adjust as needed
vmin = -vmax

# Plot map; reverse longitude to keep east-to-the-left astronomy convention
image = ax.pcolormesh(
    longitude[::-1], latitude, grid_map,
    vmin=vmin, vmax=vmax,
    cmap="seismic", rasterized=True,
)

# Orbit overlays (Matplotlib expects radians here)
# LMC orbiit
ax.plot(
    -lmc_orbit.galactic.l.wrap_at(180*u.deg).radian,
     lmc_orbit.galactic.b.radian,
     color='orange', lw=2, label='LMC orbit'
)

# LMC tänane asukoht
ax.scatter(
    -c_LMC.galactic.l.wrap_at(180*u.deg).radian,
     c_LMC.galactic.b.radian,
     marker='x', color='red', s=120, label='LMC today (SIMBAD)'
)

# Grid and tick labels
ax.grid(True, color="gray", lw=0.6, alpha=0.8)
ax.set_longitude_grid(60)
ax.set_latitude_grid(30)
ax.set_longitude_grid_ends(90)
ax.xaxis.set_major_formatter(ThetaFormatterShiftPi(60))
ax.tick_params(axis='x', labelsize=10, colors='black')
ax.tick_params(axis='y', labelsize=10, colors='black')

# Colorbar
cax = fig.add_axes([0.15, 0.08, 0.7, 0.03])
cb = fig.colorbar(image, cax=cax, orientation="horizontal")
cb.set_label("Overdensity δ = n/⟨n⟩ − 1", fontsize=12)
cb.solids.set_edgecolor("face")

# Axis labels
plt.figtext(0.5, 0.02, "Galactic longitude [deg]", ha="center", fontsize=12)
plt.figtext(0.94, 0.5, "Galactic latitude [deg]",  va="center", rotation=90, fontsize=12)

plt.subplots_adjust(bottom=0.1, top=0.98, left=0.05, right=0.98)
plt.show()
